###### Import Statement

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydot
import copy
import random
from random import randrange
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import export_graphviz
from sklearn.preprocessing import LabelEncoder
from scipy import misc
from collections import OrderedDict
from operator import itemgetter
%matplotlib inline


###### Preprocessing

In [3]:
#Titanic Dataset
#https://www.kaggle.com/c/titanic/data
def get_titanic_data():
    df_titanic=pd.read_csv('titanic.csv',)
    #Remove unnecessary columns (body= Body Indentification number, Name= Name)
    df_titanic.drop(['name','body'],1,inplace=True)
    #Fill all the na  
    df_titanic.cabin.fillna('unknown',inplace=True)
    df_titanic.age.fillna(df_titanic['age'].mean(),inplace=True)
    df_titanic.fillna(0,inplace=True)
    #Covert nonnumeric value into numeric
    df_titanic['sex'] = LabelEncoder().fit_transform(df_titanic['sex'])
    df_titanic['cabin'] = LabelEncoder().fit_transform(df_titanic['cabin'].astype(str))
    df_titanic['embarked'] = LabelEncoder().fit_transform(df_titanic['embarked'].astype(str))
    df_titanic['home.dest'] = LabelEncoder().fit_transform(df_titanic['home.dest'].astype(str))
    df_titanic['ticket'] = LabelEncoder().fit_transform(df_titanic['ticket'])
    df_titanic['boat'] = LabelEncoder().fit_transform(df_titanic['boat'].astype(str))
    # df_titanic.head()
    # df_titanic.dtypes
    # print(df_titanic.isnull().sum())
    y = df_titanic['pclass']
    X = df_titanic.drop("pclass", axis = 1)
    return X,y

###### Train_Test_Spliting

In [4]:
#To split the dataset into 3 parts
def data_split(X_Data,y_Target):   
    X_train, X_test, y_train, y_test= train_test_split(X_Data, y_Target, test_size=0.4)
    X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size=0.5)
    return X_train,X_val,X_test,y_train,y_val,y_test

In [5]:
X,y=get_titanic_data()
X_train,X_val,X_test,y_train,y_val,y_test=data_split(X,y)

###### Methods and submethods declaration for genetic algorithm optimization

In [6]:
#Method used to split the initial random forest into random forest vector population
def partition_gene_to_chromosome(listin, n):
    random.shuffle(listin)
    return [listin[i::n] for i in range(n)]

In [7]:
#Function to determin the best fit parents
def call_population_fitness(randomForest_population):
    print(len(randomForest_population))
    fitnesslist=[]
    for rf in randomForest_population:
        fitness=rf.score(X_val,y_val)
        fitnesslist.append(fitness)
    fitness_matrix = { i : fitnesslist[i] for i in range(0, len(fitnesslist) ) }
    print(fitness_matrix)
    return fitness_matrix

In [8]:
#Fuction to select the parent population
def call_population_selection(fitness_matrix,randomForest_population,parents_number):
    fitness_ordered=sorted(fitness_matrix.items(), key=itemgetter(1),reverse=True)
    selected_chromRF_Index=list(list(zip(*fitness_ordered))[0])[:parents_number]
#     print(selected_chromRF_Index)
    selected_rf_population=[randomForest_population[i] for i in selected_chromRF_Index]
#     print("Selection",selected_rf_population)
    return selected_rf_population

In [9]:
#Sub Function for cross over
def get_offcross_spring(rf_chromosome1,rf_chromosome2,cross_over_point):
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     for i in range(len(rf_chromosome2.estimators_)):
#         print(rf_chromosome2.estimators_[i].random_state)
#     print(rf_chromosome2.estimators_,len(rf_chromosome2.estimators_),"minuuuuuuuuuuuuuuuuroseeeeeeeeeeeeeeeeeeeee")
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     for i in range(len(rf_chromosome1.estimators_)):
#         print(rf_chromosome1.estimators_[i].random_state)
#     print(rf_chromosome1.estimators_,len(rf_chromosome1.estimators_),"fousiyyaaaasaleeeeemmmmmmm")
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(cross_over_point)
    cross_1=rf_chromosome1.estimators_[0:cross_over_point]
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(cross_1,len(cross_1),"fousiyyaaaa")
    cross_2=rf_chromosome2.estimators_[cross_over_point:]
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(cross_2,len(cross_2),"minuuuuuuuuuuuuuuuu")
    modified_estimators=cross_1+cross_2
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     for i in range(len(modified_estimators)):
#         print(modified_estimators[i].random_state)
#     print(modified_estimators,"anwarrrrrrrrrrr")
    rf_chromosome1.estimators_=modified_estimators
#     print(rf_chromosome1)
    return rf_chromosome1

In [10]:
# Main Crossover Function
def call_crossover_generation(selected_rf_population,crossover_per_gen):
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(selected_rf_population[1].estimators_)
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
    offsprings_gen = []
    # The crossoverpoint
    cross_over_point = np.uint8(len(selected_rf_population[1])/2)   
    for k in range(int(crossover_per_gen)):
        # Crossoverindex of the first parent
        parent1_index = k % len(selected_rf_population)
        # Crossoverindex of the seconf parent
        parent2_index = (k+1) % len(selected_rf_population)
        offspring=get_offcross_spring(selected_rf_population[parent1_index],selected_rf_population[parent2_index],cross_over_point)
        offsprings_gen.append(offspring)
#         print(k)
#         print(offspring.estimators_)
#     print(len(offsprings_gen))
    return offsprings_gen

In [11]:
# Method to get the mutated chromosomes
def get_mutation_gen(randomForest,selected_rf_population,mutation_per_gen):
    offsprings_gen = []
    rf_for_mutation=[]
    for k in range(int(mutation_per_gen)):  
        # Randomly choose a chromosome for mutation.
        parent1_index = k % len(selected_rf_population)
        rf_for_mutation=random.choice(selected_rf_population)
        #Randomly choose one of the gene for mutation
        gene_for_mutation = random.choice(rf_for_mutation.estimators_)
        rf_for_mutation.estimators_.remove(gene_for_mutation)
        slct_impurity_df=random.choice(randomForest)
        rf_for_mutation.estimators_.append(slct_impurity_df)
        offsprings_gen.append(rf_for_mutation)
#     print(offsprings_gen[0].estimators_)
    return offsprings_gen

In [17]:
M,N=X.shape
F=int(N**(1/2))

#Total number of decision trees all random forest
initial_population=500
initial_population_list=list(range(0,initial_population))

#Total number of chromosome(No of sub random forest for genetic algorithm)
n=100
cross_over_rate=0.9
mutation_rate=0.1
number_of_gen=100
crossover_per_gen=(cross_over_rate*n)
mutation_per_gen=(mutation_rate*n)
print(crossover_per_gen)
print(mutation_per_gen)

#Number of parents for next generation
parents_number=50

# Random forest model 
# Generating first population of raandom forest
randomForest = RandomForestClassifier(n_estimators=initial_population,max_features=F)
randomForest.fit(X_train,y_train)
print(len(randomForest.estimators_))

90.0
10.0
500


In [18]:
sub_forest_chromosome=partition_gene_to_chromosome(initial_population_list,n)

In [19]:
#List of initial population (chromosomes)
randomForest_population=[]
for chrom in range(n):
    rf_classifier=RandomForestClassifier(n_estimators=len(sub_forest_chromosome[chrom]))
    rf_classifier.estimators_=[]
    for gene in sub_forest_chromosome[chrom]:
        rf_classifier.estimators_.append(randomForest.estimators_[(gene-1)])    
    rf_classifier.classes_=randomForest.classes_
    rf_classifier.n_classes_=randomForest.n_classes_
    rf_classifier.n_outputs_=randomForest.n_outputs_ 
    randomForest_population.append(rf_classifier)


In [20]:
for gen in range(number_of_gen):
    print("Generation Number: ", gen)
    # To measure the fitness of each chromosome
    fitness_matrix=call_population_fitness(randomForest_population)

    # To select the best parents for next generation
    # print("next_gen_parents\n\n\n\n\n\n\n\n\n\n",randomForest_population)
    # print("next_gen_parents decision tree                 \n\n\n\n\n\n\n\n\n\n",randomForest_population[0].estimators_)
    next_gen_parents = call_population_selection(fitness_matrix,randomForest_population,parents_number)

    # Crossover for generating offspring
    # print("crossover_offsprings\n\n\n\n\n\n\n\n\n\n",next_gen_parents)
    # print("crossover_offsprings decision tree\n\n\n\n\n\n\n\n\n\n",next_gen_parents[0].estimators_)
    crossover_offsprings = call_crossover_generation(copy.deepcopy(next_gen_parents),crossover_per_gen)
    # print(crossover_offsprings)
    # Mutation for the generated offspring
    # print(next_gen_parents)
    mutated_offspring = get_mutation_gen(randomForest,copy.deepcopy(next_gen_parents),mutation_per_gen)
    # print(mutated_offspring.estimators_)
    randomForest_population=[]
    randomForest_population=mutated_offspring+crossover_offsprings
#     print(randomForest_population[1].estimators_)
    #     # New population for next generation
    #     next_population=[]
    #     next_population.append(crossover_offsprings)
    #     next_population.append(mutated_offspring)
    #     print(len(next_population))
    #     randomForest_population=next_population


Generation Number:  0
100
{0: 0.9211195928753181, 1: 0.9491094147582697, 2: 0.9312977099236641, 3: 0.9669211195928753, 4: 0.9414758269720102, 5: 0.9363867684478372, 6: 0.910941475826972, 7: 0.9465648854961832, 8: 0.9338422391857506, 9: 0.9083969465648855, 10: 0.9516539440203562, 11: 0.9541984732824428, 12: 0.9516539440203562, 13: 0.9236641221374046, 14: 0.9312977099236641, 15: 0.9338422391857506, 16: 0.9440203562340967, 17: 0.9541984732824428, 18: 0.9440203562340967, 19: 0.9338422391857506, 20: 0.9567430025445293, 21: 0.9465648854961832, 22: 0.9312977099236641, 23: 0.9414758269720102, 24: 0.910941475826972, 25: 0.9643765903307888, 26: 0.9312977099236641, 27: 0.9491094147582697, 28: 0.9414758269720102, 29: 0.9643765903307888, 30: 0.9185750636132316, 31: 0.9440203562340967, 32: 0.9491094147582697, 33: 0.9491094147582697, 34: 0.9185750636132316, 35: 0.9389312977099237, 36: 0.9491094147582697, 37: 0.9465648854961832, 38: 0.9338422391857506, 39: 0.9338422391857506, 40: 0.9363867684478372, 4

Generation Number:  5
100
{0: 0.9491094147582697, 1: 0.9491094147582697, 2: 0.9516539440203562, 3: 0.9491094147582697, 4: 0.9541984732824428, 5: 0.9516539440203562, 6: 0.9720101781170484, 7: 0.9516539440203562, 8: 0.9541984732824428, 9: 0.9541984732824428, 10: 0.9643765903307888, 11: 0.9669211195928753, 12: 0.9669211195928753, 13: 0.9669211195928753, 14: 0.9669211195928753, 15: 0.9669211195928753, 16: 0.9669211195928753, 17: 0.9669211195928753, 18: 0.9669211195928753, 19: 0.9669211195928753, 20: 0.9669211195928753, 21: 0.9669211195928753, 22: 0.9669211195928753, 23: 0.9669211195928753, 24: 0.9669211195928753, 25: 0.9618320610687023, 26: 0.9618320610687023, 27: 0.9618320610687023, 28: 0.9618320610687023, 29: 0.9618320610687023, 30: 0.9618320610687023, 31: 0.9618320610687023, 32: 0.9618320610687023, 33: 0.9618320610687023, 34: 0.9618320610687023, 35: 0.9618320610687023, 36: 0.9618320610687023, 37: 0.9618320610687023, 38: 0.9618320610687023, 39: 0.9618320610687023, 40: 0.9618320610687023,

{0: 0.9414758269720102, 1: 0.9414758269720102, 2: 0.9414758269720102, 3: 0.9414758269720102, 4: 0.9567430025445293, 5: 0.9414758269720102, 6: 0.9414758269720102, 7: 0.9414758269720102, 8: 0.9414758269720102, 9: 0.9414758269720102, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349, 41: 0.9745547073791349, 4

{0: 0.9338422391857506, 1: 0.9338422391857506, 2: 0.9338422391857506, 3: 0.9338422391857506, 4: 0.9338422391857506, 5: 0.9338422391857506, 6: 0.9338422391857506, 7: 0.9338422391857506, 8: 0.9338422391857506, 9: 0.9338422391857506, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349, 41: 0.9745547073791349, 4

Generation Number:  18
100
{0: 0.9541984732824428, 1: 0.9541984732824428, 2: 0.9541984732824428, 3: 0.9541984732824428, 4: 0.9541984732824428, 5: 0.9541984732824428, 6: 0.9541984732824428, 7: 0.9541984732824428, 8: 0.9541984732824428, 9: 0.9541984732824428, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349

...

{0: 0.9363867684478372, 1: 0.9363867684478372, 2: 0.9363867684478372, 3: 0.9363867684478372, 4: 0.9363867684478372, 5: 0.9363867684478372, 6: 0.9363867684478372, 7: 0.9363867684478372, 8: 0.9363867684478372, 9: 0.9363867684478372, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349, 41: 0.9745547073791349, 4

{0: 0.9618320610687023, 1: 0.9618320610687023, 2: 0.9618320610687023, 3: 0.9618320610687023, 4: 0.9618320610687023, 5: 0.9618320610687023, 6: 0.9618320610687023, 7: 0.9618320610687023, 8: 0.9618320610687023, 9: 0.9618320610687023, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349, 41: 0.9745547073791349, 4

{0: 0.9338422391857506, 1: 0.9338422391857506, 2: 0.9338422391857506, 3: 0.9338422391857506, 4: 0.9338422391857506, 5: 0.9338422391857506, 6: 0.9338422391857506, 7: 0.9338422391857506, 8: 0.9338422391857506, 9: 0.9338422391857506, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349, 41: 0.9745547073791349, 4

Generation Number:  93
100
{0: 0.9440203562340967, 1: 0.9440203562340967, 2: 0.9440203562340967, 3: 0.9440203562340967, 4: 0.9440203562340967, 5: 0.9440203562340967, 6: 0.9440203562340967, 7: 0.9440203562340967, 8: 0.9440203562340967, 9: 0.9440203562340967, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349

{0: 0.9618320610687023, 1: 0.9618320610687023, 2: 0.9618320610687023, 3: 0.9618320610687023, 4: 0.9618320610687023, 5: 0.9618320610687023, 6: 0.9618320610687023, 7: 0.9618320610687023, 8: 0.9618320610687023, 9: 0.9618320610687023, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349, 41: 0.9745547073791349, 4

In [21]:
fitness_matrix=call_population_fitness(randomForest_population)
next_gen_parents = call_population_selection(fitness_matrix,randomForest_population,parents_number)
print(len(next_gen_parents))
fitness_matrix=call_population_fitness(next_gen_parents)
parent_num=[1,2,4,10]
final_gen_parents=[]
for i in range(len(parent_num)):    
    final_gen_parents_ = call_population_selection(fitness_matrix,randomForest_population,parent_num[i])
    final_gen_parents.append(final_gen_parents_)
    
print(final_gen_parents)

final_estimators=[]
for i in range(len(parent_num)):
    estimator=[]
    randomforest_list=final_gen_parents[i]
    for j in range(parent_num[i]):
       
        estimator.extend(randomforest_list[j].estimators_)
            
    final_estimators.append(estimator)

100
{0: 0.9211195928753181, 1: 0.9211195928753181, 2: 0.9211195928753181, 3: 0.9211195928753181, 4: 0.9211195928753181, 5: 0.9211195928753181, 6: 0.9211195928753181, 7: 0.9211195928753181, 8: 0.9211195928753181, 9: 0.9211195928753181, 10: 0.9745547073791349, 11: 0.9745547073791349, 12: 0.9745547073791349, 13: 0.9745547073791349, 14: 0.9745547073791349, 15: 0.9745547073791349, 16: 0.9745547073791349, 17: 0.9745547073791349, 18: 0.9745547073791349, 19: 0.9745547073791349, 20: 0.9745547073791349, 21: 0.9745547073791349, 22: 0.9745547073791349, 23: 0.9745547073791349, 24: 0.9745547073791349, 25: 0.9745547073791349, 26: 0.9745547073791349, 27: 0.9745547073791349, 28: 0.9745547073791349, 29: 0.9745547073791349, 30: 0.9745547073791349, 31: 0.9745547073791349, 32: 0.9745547073791349, 33: 0.9745547073791349, 34: 0.9745547073791349, 35: 0.9745547073791349, 36: 0.9745547073791349, 37: 0.9745547073791349, 38: 0.9745547073791349, 39: 0.9745547073791349, 40: 0.9745547073791349, 41: 0.974554707379134

#### Final Result of the optmized dataset

In [32]:
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import zero_one_loss
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


iterations=[5,10,20,50] #" Iterations ",
F1_score=[]
Precision_score=[]
Recall_score=[]
Accuracy_Score=[]#" Iterations ",
Error_Score=[]#" Error "
Prediction_time=[]#" Prediction Time "
for i in range(len(iterations)):
    print(final_estimators[i],"\n\n\n\n\n\n\n\nfinal_estimators[i]")
    new_randomForest_subSet=RandomForestClassifier(n_estimators=len(final_estimators[i]))
    new_randomForest_subSet.estimators_=final_estimators[i]
    new_randomForest_subSet.classes_=randomForest.classes_
    new_randomForest_subSet.n_classes_=randomForest.n_classes_
    new_randomForest_subSet.n_outputs_=randomForest.n_outputs_
    start = time.clock()    
    print(new_randomForest_subSet)
    print(X_test)
    y_pred=new_randomForest_subSet.predict(X_test)
    end = time.clock()
    Accuracy_Sc=accuracy_score(y_test,y_pred)
    error_Sc=zero_one_loss(y_test,y_pred)
    Accuracy_Score.append(Accuracy_Sc)
    Error_Score.append(error_Sc)

    f1_score_=f1_score(y_test, y_pred, average="macro")
    precision_score_=precision_score(y_test, y_pred, average="macro")
    recall_score_= recall_score(y_test, y_pred, average="macro")

    F1_score.append(f1_score_)
    Precision_score.append(precision_score_)
    Recall_score.append(recall_score_)

    time1=(end-start)
    Prediction_time.append(time1)




iterations = pd.DataFrame(np.transpose([iterations]),columns=["Cluster_no/DecsionTree_No"])
Accuracy_Score= pd.DataFrame(np.transpose([Accuracy_Score]),columns=["Accuracy_Score"])
Error_Score= pd.DataFrame(np.transpose([Error_Score]),columns=["Error_Score"])
F1_score= pd.DataFrame(np.transpose([F1_score]),columns=["F1_score"])
Precision_score= pd.DataFrame(np.transpose([Precision_score]),columns=["Precision_score"])
Recall_score= pd.DataFrame(np.transpose([Recall_score]),columns=["Recall_score"])

Prediction_tim= pd.DataFrame(np.transpose([Prediction_time]),columns=["Prediction_time"])        
dataa=pd.concat([iterations,Accuracy_Score,Error_Score,Prediction_tim],axis=1)




[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=66381665, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1028177874, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.

C:\Users\Petrofac\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\Petrofac\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\Petrofac\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\Petrofac\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\Petrofac\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: time.clo

In [33]:
print(dataa)

   Cluster_no/DecsionTree_No  Accuracy_Score  Error_Score  Prediction_time
0                          5        0.921756     0.078244         0.018364
1                         10        0.921756     0.078244         0.016743
2                         20        0.921756     0.078244         0.020259
3                         50        0.921756     0.078244         0.022541
